# MEEPART 2D Demo : Adding and studying details and defects

Welcome to the guide of MEEPART : MEEP for the Analysis of Refractor Telescopes Part 2, which follows the first part presented in MEEPART_GUIDE. In this part we will add defects to the parts presented previously and new components, such as a telescope tube with absorbers and a half wave plate.

**This guide is only for the 2D version of MEEPART.**

**Resolution plays an important role in the convergence of the far field beams. Results shown here are shown for pedagogical purposes, resolution is set to a lower value to reduce computation rime. Do not hesitate to investigate the effect of resolution on your results.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep_optics as mo

We will keep the same example system as seen in Part 1.

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300,
                     x = 140.)

lens2 = mo.AsphericLens(name = 'Lens 2', 
                     r1 = 269.190, 
                     r2 = 6398.02, 
                     c1 = -2.4029, 
                     c2 = 1770.36,
                     diameter = 300,
                     thick = 40, 
                     x = 549.408)
aperture_stop = mo.ApertureStop(name = 'Aperture Stop',
                                 pos_x = 10,
                                 diameter = 200,
                                 thickness = 2,
                                 n_refr = 1.1, 
                                 conductivity = 1e7)
image_plane = mo.ImagePlane(name = 'Image Plane',
                         pos_x = 10+714.704,
                         diameter = 300,
                         thickness = 2,
                         n_refr = 1.1, 
                         conductivity = 0.01)

Let's make a function that makes all the system assembly.

In [ ]:
def system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,300]):
    opt_sys = mo.OpticalSystem('Example Telescope')
    opt_sys.set_size(opt_sys_size[0],opt_sys_size[1])
    
    for component in components :
        opt_sys.add_component(component)
    
    opt_sys.assemble_system(dpml = dpml, res = resolution)
    opt_sys.write_h5file()
    return opt_sys

We are now ready to add details to the system.

**Material properties:** The first kind of defect can be an alteration of the material properties, which can be easily studied, by defining the objects as previously shown and changing their properties (index and conductivity). The lens shape can also be finely changed. Keep in mind that small changes in the geometry need a high resolution to be represented and will very probably only have an effect at smaller wavelengths.

For example, if you want to change the index of refraction of the lenses, you can just specify it in the definition of the lenses with the n_refr attribute : 

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300,
                     x = 140.,
                     n_refr = 1.52)

**Anti-Reflection (AR) coating:** It is possible to add AR coating on each face of each lens independently, as shown below.

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300, 
                     x = 140.,
                    AR_left = 2.5,
                    AR_right = 2.5)

Or equivalently :

In [ ]:
lens1.AR_left = 2.5
lens2.AR_right = 2.5

The AR coating permittivity is automatically set to be the square root of the permittivity of the lenses. This means that the permittivity of the AR coating is equal to the index of refraction of the lens. You can however change that by specifying a new permittivity.

In [ ]:
lens1.AR_material = 1.52 #Keep in mind that you specify the permittivity here, which is the square of the index.

AR coating is not a defect, however AR coating delamination is. It is modeled as lumps of air randomly placed on the surface of the lenses, of which we can control the width and thickness.

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300,
                     x = 140.,
                     AR_left = 2.5,
                     AR_right = 2.5,
                     delam_thick = 1,
                     delam_width = 5)

#OR 

lens1.delam_thick = 1
lens2.delam_width = 5

Recall that everything is specified in the units that you have chosen. If it is the mm, the delamination set in the above example will show as a lump on each side of the lens of a 1mm air gap and 5mm wide.

Let's look at the effect of delamination on the far field beam. Recall that small defects might not have an effect on large wavelengths, however smaller wavelengths require more computation time. For this example we will keep relatively large wavelengths compared to the defects so it is possible to run the examples within a reasonable amount of time.

First, define a function that does the analysis to make things shorter.

In [ ]:
def run_analysis(opt_sys, resolution = 2, wavelength = 10):
    sim = mo.Sim(opt_sys)
    sim.set_verbosity(0)
    analysis = mo.Analysis(sim)
    analysis.image_plane_beams(wvl = wavelength, simres = 2, runtime = 800, beam_w0 = 10)
    freq, beam = analysis.beam_FT()
    return freq, beam, analysis

Define the wavelength, PML thickness and resolution (recall that PML thickness is at least half a wavelength and wavelength times resolution needs to be at least 8 in the highest index of refraction of the system)

In [ ]:
wavelength = 10
dpml = 5
resolution = 2

Now we can compare the case with delamination and the case without delamination.

In [ ]:
#Add AR coating
lens1.AR_left = 2
lens1.AR_right = 2
lens2.AR_left = 2
lens2.AR_right = 2

#Reset the delamination on the 1st lens :
lens1.delam_thick = 0

components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution)
freq, beam, analysis = run_analysis(opt_sys)

#Compare with delamination
lens1.delam_thick = 1
lens1.delam_width = 20
lens2.delam_thick = 1
lens2.delam_width = 20
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution)
freq, beam_delam, analysis = run_analysis(opt_sys)

beams = [beam[0], beam_delam[0]]

In [ ]:
legend = ['Without delamination', 'With delamination']
%matplotlib inline
analysis.plotting(freq, beams, wavelength, legend = legend, print_solid_angle = True, ylim = -30)

As can be seen, the effect of delamination, especially at a large wavelength compared to the air layer created is quite small. Of course, one could also play with the AR coating thickness and index to look at the effect of such changes, however it is not expected to be a very strong effect. Recall that the beams are all normalized, and cannot represent a more efficient transimission.

**Gradient in the index of refraction of the lenses:** It is possible to put an overall radial and/or axial gradient in the index of refraction of the lens. This is done with the radial_slope and axial_slope arguments as shown below. Their value is defined in (units of MEEP length)^(-1). The axial slope is defined to keep the nominal index value at the front of the lens, and the radial gradient at the center of the lens. This means that an axial_slope of 0.01 will give an index change of 0.5 at the back of a 50 units thick lens. A radial_slope of 0.01 will give an index change of 1 on the outer radius of a 200mm diameter lens.

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300,
                     x = 140.,
                     radial_slope = 1e-3)

#OR 

lens1.radial_slope = 2e-3
lens2.radial_slope = 2e-3

Let's look at the effect of a radial gradient.

In [ ]:
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution)
freq, beam, analysis = run_analysis(opt_sys)

#Set the gradient back to 0
lens1.radial_slope = 0
lens2.radial_slope = 0
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution)
freq, beam_gradient, analysis = run_analysis(opt_sys)

#Plot
beams = [beam[0], beam_gradient[0]]
legend = ['With gradient', 'Without gradient']
analysis.plotting(freq, beams, wavelength, legend = legend, print_solid_angle = True, ylim = -40)

In the example shown above, it results in a change in the index of refraction of 0.3, which is quite radical, but shows well the effect.

**Clusters of air bubbles in the lenses:** When lenses are made out of plastic and through the process of molding, there can be bubbles of air in the material. To study that, MEEPART provides three parameters to a function creating randomly placed clusters of bubbles. These parameters are:
- radius
- nb_clusters
- nb_per_cluster
The way each cluster is generated is by placing a main bubble of size 'radius' +/-10% randomly inside the lens (with a condition on placement, not too close to the border of the lens so that bubbles do not show up as craters on the surface). Then 'nb_per_cluster' additional bubbles are added around the main bubble, so that their size decreases with distance from this bubble. 'nb_clusters' controls the number of clusters per lens. 

Including bubbles requires to change the assembly function, as it is a separate function that does this.

In [ ]:
def system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,300],
                    radius = 0, nb_clusters = 0, nb_per_cluster = 0):
    opt_sys = mo.OpticalSystem('Example Telescope')
    opt_sys.set_size(opt_sys_size[0],opt_sys_size[1])
    
    for component in components :
        opt_sys.add_component(component)
    
    opt_sys.assemble_system(dpml = dpml, res = resolution)
    opt_sys.make_lens_bubbles(radius, nb_clusters, nb_per_cluster)
    opt_sys.write_h5file()
    return opt_sys

Again, let's look at the effect this has on the beam.

In [ ]:
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution)
freq, beam, analysis = run_analysis(opt_sys)

#Add bubbles
opt_sys = system_assembly(components, wavelength, dpml, resolution, radius = 2, nb_clusters = 2, nb_per_cluster = 4)
freq, beam_gradient, analysis = run_analysis(opt_sys)

#Plot
beams = [beam[0], beam_gradient[0]]
legend = ['Without bubbles', 'With bubbles']
analysis.plotting(freq, beams, wavelength, legend = legend, print_solid_angle = True, ylim = -40)

**Telescope tube and absorbers:** An important source of systematics in telescopes is the reflections of light on the telescope tube. It is possible to add a perfect conductor tube and an absorbing layer on top of it to minimize reflections on the said tube.

Adding the tube, we provide the center of the slab along the y-axis. The lenses have a radius of 150 here, we add 5 for the absorbing layer and half a thickness of the tube which gives 157.5 for the center position on y with a thickness of 5.

In [ ]:
tube = TelescopeTube(thick = 10, center = 165, name = 'Tube')

Adding the absorbing layer, the thickness and center are provided as shown for the tube. The material properties are the real and imaginary part of the material permittivity as well as the studied frequency (it is not a property of the material in itself but is required to properly define the imaginary part of the permittivity within Meep).

In [ ]:
absorber = Absorber(thick = 10, center = 155, epsilon_real = 3.5, epsilon_imag = 0.05, freq = 1/wavelength, name = 'Absorber')

Now let's compare the situation with and without an absorber. Remember to give a higher size along y to the system in order to fit the tube.

In [ ]:
components = [lens1, lens2, image_plane, aperture_stop, tube]
opt_sys = system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,340])
freq, beam, analysis1 = run_analysis(opt_sys)

#Compare with and without absorber delamination
components = [lens1, lens2, image_plane, aperture_stop, tube, absorber]
opt_sys = system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,340])
freq, beam_absorb, analysis2 = run_analysis(opt_sys)

#And without tube
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,340])
freq, beam_raw, analysis3 = run_analysis(opt_sys)

In [ ]:
beams = [beam[0], beam_absorb[0], beam_raw[0]]
legend = ['Without absorber', 'With absorber', 'Without tube']
analysis2.plotting(freq, beams, wavelength, legend = legend, print_solid_angle = True, ylim = -25)

Note : It is a good idea to sometimes plot the field (with analysis.sim.plot_efield()) to check that all objects are at the right place or even just to see how reflections happen when there is a reflective part in the system.

The effect of a tube would be more visible with scattering of light on the lens surfaces, it is possible to have a look at that. To correctly model surface errors, it is best to increase resolution.
The surface errors are modeled as patches of fixed width with same offset from the initial perfect surface, and whose distribution of offsets follows a gaussian of scale surf_err_scale.

In [ ]:
lens1.surf_err_width = 5
lens1.surf_err_scale = 0.5
lens2.surf_err_width = 5
lens2.surf_err_scale = 0.5

components = [lens1, lens2, image_plane, aperture_stop, tube]
opt_sys = system_assembly(components, wavelength, dpml, 4, opt_sys_size = [800,340])
freq, beam, analysis1 = run_analysis(opt_sys, resolution = 4)

#Compare with and without absorber delamination
components = [lens1, lens2, image_plane, aperture_stop, tube, absorber]
opt_sys = system_assembly(components, wavelength, dpml, 4, opt_sys_size = [800,340])
freq, beam_absorb, analysis2 = run_analysis(opt_sys, resolution = 4)

#And without tube
components = [lens1, lens2, image_plane, aperture_stop]
opt_sys = system_assembly(components, wavelength, dpml, 4, opt_sys_size = [800,340])
freq, beam_raw, analysis3 = run_analysis(opt_sys, resolution = 4)

In [ ]:
beams = [beam[0], beam_absorb[0], beam_raw[0]]
legend = ['Without absorber', 'With absorber', 'Without tube']
analysis2.plotting(freq, beams, wavelength, legend = legend, print_solid_angle = True, ylim = -25)
analysis3.sim.plot_efield()

**Half-Wave Plate (HWP):** A HWP can be added to the system. It has to obey the following equation

$\Gamma = \frac{2 \pi \Delta n L}{\lambda_0} = \pi$

which constrains the thickness of the plate $L$ and difference between the fast axis and slow axis indexes $\Delta n = n_e - n_0$.

The material chosen by default for the HWP is saphire, which has $n_0 = 3.019$ and $n_e = 3.336$. At a wavelength of 7 units, with saphire, we get 11.04 for $L$.

The HWP fast axis is set at an angle $\theta$ with respect to x. At $\theta = \frac{\pi}{4}$ the e field polarized along $\vec{e_z}$ becomes polarized along $\vec{e_y}$.

Let's put such a HWP before the aperture.

In [ ]:
HWP = HalfWavePlate(name = 'HWP', 
                   fast_ax_index = 3.019,
                   slow_ax_index = 3.336,
                   thick = 11.04, 
                   x_pos = 70,
                   size_y = 300,
                   theta = 45)

In [ ]:
components = [lens1, lens2, image_plane, aperture_stop, HWP]
opt_sys = system_assembly(components, wavelength, dpml, resolution, opt_sys_size = [800,300])
freq, beam, analysis = run_analysis(opt_sys)

Let's look at the electric field, which is plotted by default for its Ez component. We can see an attenuation after the HWP. 

In [ ]:
analysis.sim.plot_efield()

Now look at the Ey component, which is "created" in the HWP.

In [ ]:
analysis.sim.plot_efield(comp = 'Ey')